In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models

In [24]:
master_path = '../Data/ProcessedData/train/'
dataset = torchvision.datasets.DatasetFolder(master_path, loader=torch.load, extensions=('.tensor'))

batch_size = 1
num_workers = 1
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

dataiter = iter(test_loader)
img, labels = dataiter.next()
print(img.shape)
print(labels.shape)

# require the number of channels
img = img.unsqueeze(1)
print(img.shape)

# increase the number of channels to rgb (what resnet expects)
img = img.repeat(1, 3, 1, 1)
print(img.shape)

torch.Size([1, 48, 48])
torch.Size([1])
torch.Size([1, 1, 48, 48])
torch.Size([1, 3, 48, 48])


In [9]:
resnet50 = models.resnet50()

In [10]:
# maybe need to use a normalized tensor here
# don't need to use .float() if normalized
features = resnet50(img.float())

# flattened the image already
features.shape

torch.Size([1, 1000])

In [27]:
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.name = 'emotion_classifier'
        self.fc1 = nn.Linear(1000, 100)
        self.fc2 = nn.Linear(100, 7)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        # assuming we're going to use CrossEntropyLoss, we don't need softmax here
        return x

In [28]:
emotionClassifier = EmotionClassifier()
out = emotionClassifier(features)

# apply classification on this!
print(out.shape)
print(out)

torch.Size([1, 7])
tensor([[ 0.1354,  0.0123,  0.0318, -0.1043,  0.1115,  0.2784, -0.2744]],
       grad_fn=<AddmmBackward>)
